## We need the optimal strategy to load/save models

In [3]:
# https://pytorch-lightning.readthedocs.io/en/0.8.5/weights_loading.html

# import torch

# from fande.predict import PredictorASE
# from fande.ase import FandeCalc


# # fdm = prepare_data(hparams, soap_params, traj_sample_rate=1)

# # train_data_loaders = sample_data(fdm, N_samples=14_000)

# # AG_force_model = prepare_model(fdm, train_data_loaders, hparams, soap_params, 200, 0.01, gpu_id=gpu_id)

# # test_performance(hparams, soap_params, AG_force_model, fdm) # check if working?

# # predictor = PredictorASE(
# #                 fdm,
# #                 AG_force_model,
# #                 hparams,
# #                 soap_params
# # )

# predictor_loaded = torch.load("/data1/simulations/ML_models/predictor.pt")


import torch
from fande.ase import FandeCalc

predictor_loaded = torch.load("/data1/simulations/ML_models/predictor.pt")
fande_calc = FandeCalc(predictor_loaded)

atoms = predictor_loaded.fdm.traj_train[0].copy()
atoms.calc = fande_calc

print( atoms.get_potential_energy(), atoms.get_forces() )
# 

Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  1.52it/s]

0.0 [[ 3.23001528 -1.33135843 -1.74239302]
 [-0.546278   -1.22240639  0.35187721]
 [-0.1907053  -1.89948273 -0.81701851]
 ...
 [ 0.48821259 -1.30752563  0.28121185]
 [ 0.23548126  0.74869919  1.14985275]
 [ 0.92536926 -1.72566223 -0.75263214]]


In [2]:
fande_calc.save_predictor("/data1/simulations/ML_models/predictor_just_a_copy.pt")

Saving predictor requires humongous amount of memory! Spare some dozens of GBs!


In [28]:
predictor_loaded.fdm.traj_train[0].copy()

Atoms(symbols='C192H300N12O12Si12', pbc=True, cell=[[13.752, 0.0, 0.0], [-2.14253011477129, 19.18875787296557, 0.0], [-2.0842458005893625, -8.862442591006795, 19.1333936290412]], momenta=...)

In [6]:
fande_calc = FandeCalc(predictor_loaded)

In [8]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/2023-10-19_15%3A13%3A19.421698/md_trajectory.traj", index=":")
snap = traj_295[0].copy()
snap.calc = fande_calc

In [9]:
%%capture c
# ic(traj_295[0].get_potential_energy())

from tqdm import tqdm

f_295 = []
f_fande = []
f_fande_uncertainty = []

for i in tqdm(range(4900,5000)):
        snap = traj_295[i]
        snap_copy = snap.copy()
        snap_copy.calc = fande_calc
        f_fande.append(snap_copy.get_forces())
        f_fande_uncertainty.append( snap_copy.calc.get_forces_variance(snap_copy) )
        f_295.append(snap.get_forces())

In [16]:
import os
print( os.getcwd() )
os.chdir("/data1/simulations/ML_models/trash")

/data1/simulations/ML_models/trash


In [17]:
import matplotlib.pyplot as plt
import numpy as np

symbols = snap.get_chemical_symbols()

f_295 = np.array(f_295)
f_fande = np.array(f_fande)

error = np.abs(f_295-f_fande)

f_fande_uncertainty = np.array(f_fande_uncertainty)


os.makedirs("images", exist_ok=True)

for ind in range(528):

        fig, axs = plt.subplots(4, 1, figsize=(10, 10), sharex=True)

        # ind = 70
        plt.title("Atom " + str(ind) + ", " + symbols[ind])

        axs[0].plot(f_295[:,ind,0], label="DFT 295 K")
        axs[0].plot(f_fande[:,ind,0], label="FANDE trained on 2000 K", linestyle=":")
        
        axs[1].plot(f_295[:,ind,1], label="DFT 295 K")
        axs[1].plot(f_fande[:,ind,1], label="FANDE trained on 2000 K", linestyle=":")
        
        axs[2].plot(f_295[:,ind,2], label="DFT 295 K")
        axs[2].plot(f_fande[:,ind,2], label="FANDE trained on 2000 K", linestyle=":")
        # plt.show()

        axs[3].plot(error[:,ind,0], label="Error")
        axs[3].plot(f_fande_uncertainty[:,ind,0], label="Uncertainty", linestyle="--")
        plt.legend()
        plt.savefig("images/fande_forces_" + str(ind) + ".png")
        plt.close()